In [6]:
#import Subreddit_lists
import praw
import Api_ID
from datetime import date
import logging
import requests
import progressbar
import urllib


In [7]:
reddit = praw.Reddit(
    client_id=Api_ID.App_id,
    client_secret=Api_ID.Secret,
    password="Zuccjuicer",
    user_agent="PRAW:Link Grabbing bot by u/MetaNotFacebook",
    username="MetaNotFacebook",
)

Version 7.4.0 of praw is outdated. Version 7.6.0 was released Tuesday May 10, 2022.


In [8]:
Cute=[
'aww',
'cats',
'animalsbeingjerks',
'animalsbeingbros',
'Awwducational',
'dogs',
'corgi',
'thisismylifenow',
'blep',
'eyebleach',
'tippytaps',
'awww',
'babycorgis',
'rarepuppers'
]
Unexpected=["Unexpected"]
People_positive=[
"HumansBeingBros",
"MadeMeSmile",
"ContagiousLaughter",
"nonononoyes"
]
People_negative=[
"Whatcouldgowrong",
"WatchPeopleDieInside",
"Instant_regret"
"IdiotsInCars"
]
Misc=[
"WoahDude",
"Interestingasfuck",
"Damnthatsinteresting"

]
Funny=["funny","ContagiousLaughter"]
SubredditList=[Cute,Unexpected,People_negative,People_positive,Misc,Funny]


So it looks like reddit wont give you the video and audio together, so thats gonna need combining
video is  (v.reddit/jsdnfkj) URL + DASH_480.mp4?source=fallback
UPDATE - got it to work, requests.head... whatever that does but allowing redirects and copying redirected url to a variable.
audio is  TRICKIER IT SEEMS 
sometimes (v.reddit/sdujfhd) URl + DASH_audio.mp4    https://v.redd.it/x228x4b25h491/DASH_audio.mp4?source=fallback
othertimes https://v.redd.it/z08avb339n801/audio?source=fallback
maybe other ways too
audio is handled weirdly, but most annoyingly reddit straight up refuses v.redd.it/,,/.json
that just fails
but this method seems to sorta work but resolution is the problem
the url plus the DASH_res i need to guess the resolution ig. not too challenging. start at 1080, try and except to 720,480,
otherwise continue but there must be a better way than this

In [ ]:
pbar=None
def download(url,folder,filename):
    
    short_url = url
    orig_url = requests.head(short_url, allow_redirects=True)
    url=orig_url.url
    
    #VIDEO URL auto gets the best quality uploaded
    if url[-1]=='/':  #dk if this is necessary just trims the last / from url before a .json is concatenated
        url=url.rstrip('/')
        
    Headers={"User-Agent":"PRAW:Link Grabbing bot by u/MetaNotFacebook"}
    response=requests.get(url+".json",headers=Headers)
    Video_Url=response.json()[0]['data']['children'][0]['data']['secure_media']['reddit_video']['fallback_url']
    Nsfw_filter=response.json()[0]['data']['children'][0]['data']['over_18']
    if Nsfw_filter is False:
        print(Video_Url)
        urllib.request.urlretrieve(Video_Url, f'{folder}/{filename}_Video.mp4',show_progress)
        
        
    #AUDIO URL is a slight modification of the base v.redd.it but seems to have some variations to it
    
    for 
        Audio_Url=short_url+f'DASH_{i}.mp4?source=fallback'
    urllib.request.urlretrieve(Video_Url, f'{folder}/{filename}.mp4',show_progress) 
        
    
#NOT MY CODE got from stackoverflow
def show_progress(block_num, block_size, total_size):
    global pbar
    if pbar is None:
        pbar = progressbar.ProgressBar(maxval=total_size)
        pbar.start()

    downloaded = block_num * block_size
    if downloaded < total_size:
        pbar.update(downloaded)
    else:
        pbar.finish()
        pbar = None        

In [9]:
today = date.today()

# dd/mm/YY
Date = today.strftime("%d_%m_%Y")

In [2]:
def RetrieveAndDownload(SubredditList):
    for i in SubredditList:
        for k in i:
            c=0
            logging.info(f"Current Subreddit {k}")
            for j in reddit.subreddit(k).hot(limit=5):
                c+=1
                if j.url[8].lower()=='v':
                    print(j.url[18:31])
                    logging.info(f"Currently Downloading {c}th file for {i}")
                    #Downloader.download(url=j.url,folder=f"Donwloads/{i}",filename=f"{Date}_{c}")
                    logging.info("Download Complete")


In [20]:
RetrieveAndDownload(SubredditList)

https://v.redd.it/h53yzp7q9k491
https://v.redd.it/9l1miro2pl491
https://v.redd.it/271al3pdil491
https://v.redd.it/tgaq1ss8th491
https://v.redd.it/4ugtkw9v7i491
https://v.redd.it/h5trxopu8i491
https://v.redd.it/g039fxsqzl491
https://v.redd.it/d6tzru6uke491
https://v.redd.it/kp05vxfmmf491
https://v.redd.it/97mfwi2ipi491
https://v.redd.it/pgmwu6mdgj491
https://v.redd.it/zd2hreul7g491
https://v.redd.it/8gm12qv9ie491
https://v.redd.it/xiqks6qsde491
https://v.redd.it/z0cl9r77gj491
https://v.redd.it/326cetqtkf491
https://v.redd.it/u4wbgy2hxi491
https://v.redd.it/v9w0wfqxhk491
https://v.redd.it/fc2e8l2mqk491
https://v.redd.it/9fqa61mmrk491
https://v.redd.it/af5iqfuj6e491
https://v.redd.it/ob90c9ahwk491
https://v.redd.it/9pkjn4o1nh491
https://v.redd.it/yp0d6bsdwe491
https://v.redd.it/73ev5t3z5i491
https://v.redd.it/kx7jkpjeck491
https://v.redd.it/trkr590frj491
https://v.redd.it/rb6gtha66k491
https://v.redd.it/nzwigyatqk491
https://v.redd.it/4e22j76nvi491
https://v.redd.it/7qs93fmvkh491
https://

NotFound: received 404 HTTP response